In [1]:
import os
os.environ["SWI_HOME_DIR"] = os.path.expanduser("~/.local/swipl/lib/swipl")
os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/.local/swipl/lib/swipl/lib/x86_64-linux")
os.environ["PATH"] = os.path.expanduser("~/.local/swipl/bin") + ":" + os.environ["PATH"]

from pyswip import Prolog
from pathlib import Path
from deap import base, creator, tools, algorithms
import numpy as np
import deap 
import random


In [2]:
base_dir = Path('../symbolic_reasoning_sys')
PROLOG_FILE = base_dir / "ethics_engine_final.pl"
prolog = Prolog()
prolog.consult(PROLOG_FILE)

# Call the main decision function
query = list(prolog.query("make_decision(dropped_wallet_1, Action, Justification, Score)"))
for result in query:
    print(f"Action: {result['Action']}")
    print(f"Justification: {result['Justification']}")
    print(f"Score: {result['Score']}")

Action: return_wallet
Justification: Action return_wallet justified by self_interest ethics based on scenario context: return_wallet.
Score: 0.020000000000000004


In [3]:
GROUND_TRUTH = {
    'dropped_wallet_1': 'return_wallet',
    'dropped_wallet_2': 'return_wallet', 
    'dropped_wallet_3': 'return_wallet',
    'dropped_wallet_4': 'leave_wallet',
    'dropped_wallet_5': 'return_wallet',
    'dropped_wallet_6': 'take_wallet',
    'dropped_wallet_7': 'leave_wallet',
    'dropped_wallet_8': 'take_wallet',
}

In [4]:
def update_weights_in_file(weights):
    """
    Overwrite weight/2 facts in the Prolog file with new values.
    weights is a list or array: [w_utilitarian, w_deontological, w_self_interest]
    """
    with open(PROLOG_FILE, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if line.strip().startswith("weight(utilitarian"):
            new_lines.append(f"weight(utilitarian, {weights[0]:.3f}).\n")
        elif line.strip().startswith("weight(deontological"):
            new_lines.append(f"weight(deontological, {weights[1]:.3f}).\n")
        elif line.strip().startswith("weight(self_interest"):
            new_lines.append(f"weight(self_interest, {weights[2]:.3f}).\n")
        else:
            new_lines.append(line)

    with open(PROLOG_FILE, "w") as f:
        f.writelines(new_lines)
        

In [5]:
def run_prolog_query(weights):
    """
    Given a weight vector, update Prolog file, reload Prolog engine,
    run make_decision for all scenarios and calculate accuracy.
    """
    
    weights = np.array(weights)
    weights = weights / np.sum(weights) # Normalise weights so they sum to 1

    update_weights_in_file(weights)

    prolog = Prolog()
    prolog.consult(PROLOG_FILE)

    correct = 0
    total = len(GROUND_TRUTH)

    for scenario_name, true_action in GROUND_TRUTH.items():
        query = f"make_decision({scenario_name}, Action, Justification, Score)."
        results = list(prolog.query(query))
        if not results:
            continue
        predicted = results[0]["Action"]
        if predicted == true_action:
            correct += 1

    accuracy = correct / total
    return (accuracy,)

if __name__ == "__main__":
    test_weights = [0.33, 0.33, 0.34]
    acc = run_prolog_query(test_weights)[0]
    print(f"Accuracy for weights {test_weights}: {acc:.2f}")

Accuracy for weights [0.33, 0.33, 0.34]: 0.88


In [6]:
scenario_data = [
    ("dropped_wallet_1", ("dropped_wallet", True, True, "many_people_around")),
    ("dropped_wallet_2", ("dropped_wallet", True, True, "isolated_area")),
    ("dropped_wallet_3", ("dropped_wallet", True, False, "many_people_around")),
    ("dropped_wallet_4", ("dropped_wallet", True, False, "isolated_area")),
    ("dropped_wallet_5", ("dropped_wallet", False, True, "many_people_around")),
    ("dropped_wallet_6", ("dropped_wallet", False, True, "isolated_area")),
    ("dropped_wallet_7", ("dropped_wallet", False, False, "many_people_around")),
    ("dropped_wallet_8", ("dropped_wallet", False, False, "isolated_area")),
]

all_scenarios = {name: (event, (owner_nearby, contents_valuable, environment))
                 for name, (event, owner_nearby, contents_valuable, environment) in scenario_data}

    
def predict_action(scenario_name, weights):
    update_weights_in_file(weights)
    result = list(prolog.query(f"make_decision({scenario_name}, Action, Justification, Score)"))
    if result:
        return result[0]['Action'], result[0]['Justification'], result[0]['Score']
    else:
        return None, None, None
        
# === DEAP SETUP ===

if not hasattr(creator, "FitnessMax"):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if not hasattr(creator, "Individual"):
    creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.uniform, 0, 1) # Individuals are 3 floating-point numbers between 0 and 1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("evaluate", run_prolog_query)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.5)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(42)
    population = toolbox.population(n=50)
    NGEN = 20
    CXPB = 0.5  # crossover prob
    MUTPB = 0.3  # mutation prob

    print("Start of evolution")
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, NGEN + 1):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values, child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate invalid fitnesses
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid))
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        population[:] = offspring

        top = tools.selBest(population, 1)[0]
        normalized_weights = np.round(np.array(top) / np.sum(top), 3)
        print(f"Gen {gen}: Best Accuracy = {top.fitness.values[0]:.2f} | Weights = {normalized_weights}")

    best_ind = tools.selBest(population, 1)[0]
    print("\nBest individual:")
    print(f"Weights: Utilitarian={best_ind[0]:.3f}, Deontological={best_ind[1]:.3f}, Self-interest={best_ind[2]:.3f}")
    print(f"Accuracy: {best_ind.fitness.values[0]:.3f}")
    print(best_ind)
    for scenario in all_scenarios:
        action = predict_action(scenario, best_ind)
        rules = list(prolog.query(f"rule_sources({scenario}, {action}, Sources)."))
        print(f"Scenario: {scenario}, Action: {action}, Rules: {rules}")

if __name__ == "__main__":
    main()

Start of evolution
Gen 1: Best Accuracy = 0.88 | Weights = [0.188 0.148 0.663]
Gen 2: Best Accuracy = 0.88 | Weights = [ 0.04  -0.103  1.064]
Gen 3: Best Accuracy = 0.88 | Weights = [0.299 0.062 0.639]
Gen 4: Best Accuracy = 0.88 | Weights = [0.168 0.029 0.803]
Gen 5: Best Accuracy = 0.88 | Weights = [0.171 0.107 0.721]
Gen 6: Best Accuracy = 0.88 | Weights = [0.341 0.006 0.652]
Gen 7: Best Accuracy = 0.88 | Weights = [-0.619  0.063  1.556]
Gen 8: Best Accuracy = 0.88 | Weights = [ 0.162 -0.414  1.252]
Gen 9: Best Accuracy = 0.88 | Weights = [0.056 0.291 0.654]
Gen 10: Best Accuracy = 0.88 | Weights = [0.146 0.26  0.594]
Gen 11: Best Accuracy = 0.88 | Weights = [-0.171  0.203  0.968]
Gen 12: Best Accuracy = 0.88 | Weights = [0.007 0.148 0.845]
Gen 13: Best Accuracy = 0.88 | Weights = [0.015 0.375 0.61 ]
Gen 14: Best Accuracy = 0.88 | Weights = [-2.115 -0.48   3.595]
Gen 15: Best Accuracy = 0.88 | Weights = [-0.936 -0.005  1.941]
Gen 16: Best Accuracy = 0.88 | Weights = [0.338 0.018 0.6

In [7]:
print(run_prolog_query([0.8, 0.1, 0.1]))  # Mostly utilitarian
print(run_prolog_query([0.1, 0.8, 0.1]))  # Mostly deontological
print(run_prolog_query([0.1, 0.1, 0.8]))  # Mostly self-interest


(0.625,)
(0.625,)
(0.875,)


In [8]:
7/8

0.875